<a href="https://colab.research.google.com/github/Szibill6/Szibill6.github.io/blob/main/bmi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Hello World")

Hello World


In [ ]:
from IPython import get_ipython
from IPython.display import display

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from google.colab import files


# 1. CSV fájl feltöltése
def upload_csv():
    """Fájl feltöltése Google Colab-ba és betöltése Pandas DataFrame-be."""
    print("Kérlek, töltsd fel a CSV fájlt (Age, Height, Weight, Bmi, BmiClass oszlopokkal)!")
    uploaded = files.upload()  # Fájl feltöltése
    file_name = list(uploaded.keys())[0]
    data = pd.read_csv(file_name, decimal='.')  # Tizedes elválasztó karakter megadása
    data['Height'] = data['Height'] * 100
    # 1. Oszlopok konvertálása numerikus adattípusra
    for col in ['Age', 'Height', 'Weight', 'Bmi']:
        data[col] = pd.to_numeric(data[col], errors='coerce').astype(float)
    # Kerekítés 0 tizedesjegyig:
    for col in ['Age', 'Height', 'Weight', 'Bmi']:
        data[col] = data[col].round(0).astype(int)
    # BMI oszlophoz rendelt BMIclass javítása
    # BMI és BmiClass konzisztencia ellenőrzése és javítása
    bmi_class_mapping = data.groupby('Bmi')['BmiClass'].agg(lambda x: x.mode()[0]).to_dict()
    data['BmiClass'] = data['Bmi'].map(bmi_class_mapping)
    # Tábla visszaadása ellenőrzésre
    display(data) # Kiírjuk a táblát
    # BMI index kiszámítása és hozzáadása az adathalmazhoz
    data['BMI'] = data['Weight'] / ((data['Height'] / 100) ** 2)
    return data

# 2. Modell betanítása
def train_model(data):
    """Neurális háló betanítása a megadott adatok alapján."""
    # Adatok előkészítése
    label_encoder = LabelEncoder()
    data['BMIClassEncoded'] = label_encoder.fit_transform(data['BmiClass'].astype(str))
    num_classes = len(label_encoder.classes_)

    # BMI index használata bemenetként
    X = data[['BMI']].values
    y = data['BMIClassEncoded'].values

    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

    # Modell létrehozása
    model = Sequential([
        Dense(16, activation='relu', input_shape=(1,)), # Bemeneti alak módosítása
        Dense(8, activation='relu'),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

    # Modell értékelése
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)
    print(f"Teszt pontosság: {test_accuracy * 100:.2f}%")

    return model, scaler, label_encoder

# 3. Felhasználói adat bekérése
def get_user_input():
    """Bekéri a felhasználótól a magasságot, testsúlyt és életkort a konzolon keresztül."""
    try:
        age = int(input("Kérlek, add meg az életkorodat (év): "))
        height = float(input("Kérlek, add meg a magasságodat (cm): "))
        weight = float(input("Kérlek, add meg a testsúlyodat (kg): "))

        if age <= 0 or height <= 0 or weight <= 0:
            raise ValueError("Az adatoknak pozitív számoknak kell lenniük!")

        # BMI index kiszámítása
        bmi = weight / ((height / 100) ** 2)
        return [[bmi]], (age, height, weight) # Bemeneti formátum módosítása

    except ValueError as e:
        print(f"Hibás adat: {e}")
        return None, None

# 4. Felhasználói adat előrejelzése
def predict_user_input(model, scaler, label_encoder, user_input, original_input):
    """Előrejelzést készít a felhasználó által megadott adatok alapján."""
    if user_input is not None:
        user_input_scaled = scaler.transform(user_input)
        prediction = model.predict(user_input_scaled)
        predicted_class = label_encoder.inverse_transform([np.argmax(prediction)])
        bmi_value = user_input[0][0]
        print(f"A számított BMI: {bmi_value:.2f}")  # Kiírjuk a BMI értéket
        print(f"Előrejelzett BMI kategória: {predicted_class[0]}")

# 5. Főprogram futtatása
if __name__ == "__main__":
    # CSV fájl feltöltése
    data = upload_csv()

    # Modell betanítása
    model, scaler, label_encoder = train_model(data)

    # Felhasználói adat bekérése
    user_input, original_input = get_user_input()

    # Előrejelzés készítése
    predict_user_input(model, scaler, label_encoder, user_input, original_input) # Átadjuk az original_input argumentumot


Kérlek, töltsd fel a CSV fájlt (Age, Height, Weight, Bmi, BmiClass oszlopokkal)!


Saving bmi.csv to bmi (22).csv


,Age,Height,Weight,Bmi,BmiClass
0,61,185,109,32,Obese Class 1
1,60,171,79,27,Overweight
2,60,155,75,31,Obese Class 1
3,60,146,36,17,Underweight
4,60,158,97,39,Obese Class 2
...,...,...,...,...,...
736,34,186,96,28,Overweight
737,44,191,107,29,Overweight
738,25,182,88,27,Overweight
739,35,188,98,28,Overweight


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


19/19 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5396 - loss: 1.7135 - val_accuracy: 0.4362 - val_loss: 1.6668
Epoch 2/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5088 - loss: 1.6148 - val_accuracy: 0.4362 - val_loss: 1.5810
Epoch 3/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5210 - loss: 1.5198 - val_accuracy: 0.4832 - val_loss: 1.5063
Epoch 4/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5319 - loss: 1.4561 - val_accuracy: 0.4899 - val_loss: 1.4387
Epoch 5/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5687 - loss: 1.3577 - val_accuracy: 0.4832 - val_loss: 1.3765
Epoch 6/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5576 - loss: 1.2879 - val_accuracy: 0.4765 - val_loss: 1.3223
Epoch 7/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5637 - loss: 1.2156 - val_accuracy: 0.4698 - val_loss: 1.2751
Epoch 8/20
19/19 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5531 - loss: 1.1815 - val_accuracy: 0.4832 - val_loss: 1.2292
Ep